# I'll be scraping five websites for PlanSmart's initial implementation, including YouTube, Coursera, Udemy, EdX, and Simplilearn.

In [5]:
!pip install vaderSentiment

                                              0.0/126.0 kB ? eta -:--:--
     -------------------------------------- 126.0/126.0 kB 3.6 MB/s eta 0:00:00


In [55]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import BertForSequenceClassification

## For Youtube

In [57]:

# Get the list of videos
url = "https://www.youtube.com/watch?v=DSoh1Iunno8"
response = requests.get(url)
response

<Response [200]>

In [58]:
soup = BeautifulSoup(response.content, "html.parser")
soup

<!DOCTYPE html>
<html darker-dark-theme="" darker-dark-theme-deprecate="" lang="en-GB" style="font-size: 10px;font-family: Roboto, Arial, sans-serif;" system-icons="" typography="" typography-spacing=""><head><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="Ash03CRZ76itNFXlh04s/Kx5TBY+uSojTNLnxUHnV26xBFl3D+0epAJQ98cRF8WLbalbrCwfmJ4GSLOYXit09g0AAABteyJvcmlnaW4iOiJodHRwczovL3lvdXR1YmUuY29tOjQ0MyIsImZlYXR1cmUiOiJQcml2YWN5U2FuZGJveEFkc0FQSXMiLCJleHBpcnkiOjE2ODgwODMxOTksImlzU3ViZG9tYWluIjp0cnVlfQ==" http-equiv="origin-trial"/><script nonce="VZfbD_EreMlt3pQGW8FZMg">var ytcfg={d:function(){return window.yt&&yt.config_||ytcfg.data_||(ytcfg.data_={})},get:function(k,o){return k in ytcfg.d()?ytcfg.d()[k]:o},set:function(){var a=arguments;if(a.length>1)ytcfg.d()[a[0]]=a[1];else{var k;for(k in a[0])ytcfg.d()[k]=a[0][k]}}};
window.ytcfg.set('EMERGENCY_BASE_URL', '\/error_204?t\x3djserror\x26level\x3dERROR\x26client.name\x3d1\x26client.version\x3d2.20230525.00.00');</script><scri

In [61]:
# class list set
class_list = set()
   
# get all tags
tags = {tag.name for tag in soup.find_all()}
  
# iterate all tags
for tag in tags:
  
    # find all element of tag
    for i in soup.find_all( tag ):
  
        # if tag has attribute of class
        if i.has_attr( "class" ):
  
            if len( i['class'] ) != 0:
                class_list.add(" ".join( i['class']))
  
print( class_list )

{'external-icon', 'menu-button skeleton-bg-color', 'video-skeleton', 'details flex-1', 'yt-icons-ext', 'thumbnail skeleton-bg-color', 'autoplay skeleton-light-border-bottom', 'masthead_shell', 'searchbox', 'skeleton-light-border-bottom', 'video-details', 'watch-main-col', 'watch-skeleton', 'flex-1', 'video-title text-shell skeleton-bg-color', 'masthead-skeleton-icon', 'masthead_custom_styles', 'text-shell skeleton-bg-color', 'global_styles', 'shell chunked', 'ytd-searchbox-spt', 'skeleton flexy', 'skeleton-bg-color', 'video-meta text-shell skeleton-bg-color', 'kevlar_global_styles'}


In [53]:
videos = soup.find_all(class_ ="video-title text-shell skeleton-bg-color")
videos

[<div class="video-title text-shell skeleton-bg-color"></div>,
 <div class="video-title text-shell skeleton-bg-color"></div>,
 <div class="video-title text-shell skeleton-bg-color"></div>,
 <div class="video-title text-shell skeleton-bg-color"></div>,
 <div class="video-title text-shell skeleton-bg-color"></div>,
 <div class="video-title text-shell skeleton-bg-color"></div>,
 <div class="video-title text-shell skeleton-bg-color"></div>,
 <div class="video-title text-shell skeleton-bg-color"></div>,
 <div class="video-title text-shell skeleton-bg-color"></div>,
 <div class="video-title text-shell skeleton-bg-color"></div>]

In [40]:
# Create a DataFrame to store the results
df = pd.DataFrame(columns=["Video Title", "Sentiment Score"])

# Loop through the videos and get the sentiment score
for video in videos:
    video_title = video.find("a", class_="ytd-video-title").text
    url = video.find("a", class_="ytd-video-title").get("href")

    # Get the comments from the video
    response = requests.get(url + "comments")
    soup = BeautifulSoup(response.content, "html.parser")
    comments = soup.find_all("div", class_="ytd-comment-renderer")

    # Analyze the comments using sentiment analysis
    analyzer = SentimentIntensityAnalyzer()
    scores = []
    for comment in comments:
        score = analyzer.polarity_scores(comment.find("span", class_="ytd-comment-text").text)["compound"]
        scores.append(score)

    # Add the sentiment score to the DataFrame
    df = df.append({"Video Title": video_title, "Sentiment Score": np.mean(scores)}, ignore_index=True)

# Print the DataFrame
print(df)


Empty DataFrame
Columns: [Video Title, Sentiment Score]
Index: []
